In [128]:
import pandas              as pd
import numpy               as np
import matplotlib.pyplot   as plt

from keras.layers          import Dense
from keras.models          import Sequential, load_model
from keras.callbacks       import EarlyStopping
from keras.metrics         import categorical_crossentropy
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler

In [101]:
# Importing train and test data

train = pd.read_csv('../data/train.zip')
test = pd.read_csv('../data/test.zip')

In [102]:
# Separating the target data from the train data
target = train.label
train.drop(columns= ['label'], axis = 1, inplace = True)

In [103]:
# Formatting the data

normalizer = MinMaxScaler(feature_range = (0, 1))

# Transform data in binary
lb = LabelBinarizer()

# Normalize input train and test data
x_train = normalizer.fit_transform(train)
x_test = normalizer.fit_transform(test)

y_train = lb.fit_transform(target)

In [148]:
def new_model(hidden_layers = 1, nodes = 10, patience = 5):

    file_name = f'no_cnn_{hidden_layers}_layer_{nodes}_nodes'

    # Instantiating model
    model = Sequential()

    # Creating early_stopping
    early_stopping_monitor = EarlyStopping(patience = patience,
                                           monitor='val_loss')

    # Adding hidden layers
    for _ in range (0, hidden_layers):
        
        model.add(Dense(nodes, activation = 'relu'))

    # Adding final layer
    model.add(Dense(10, activation = 'softmax'))

    # Compiling the model
    model.compile(optimizer = 'adam', 
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])

    # Fit the model
    model.fit(x_train, 
              y_train, 
              epochs = 100,  
              shuffle = True, 
              verbose = 2,
              callbacks = [early_stopping_monitor],
              validation_split = 0.3)

    # Creating submission file
    submission = pd.DataFrame(np.argmax(model.predict(x_test), axis = -1), columns = ['Label'])

    submission.rename_axis('ImageId', inplace=True)

    #Creating Scores Dataframe
    scores = pd.DataFrame(model.history.history)

    #Saving model
    model.save(f'../models/{file_name}.h5')

    # Saving submission file
    submission.to_csv(f'../submission/{file_name}.csv')
    
    # Saving Scores
    scores.to_csv(f'../results/{file_name}.csv')



In [149]:
# Best result so far!
new_model(hidden_layers = 3, nodes = 40)

Epoch 1/100
919/919 - 2s - loss: 0.4488 - accuracy: 0.8639 - val_loss: 0.2180 - val_accuracy: 0.9348
Epoch 2/100
919/919 - 1s - loss: 0.1943 - accuracy: 0.9400 - val_loss: 0.2019 - val_accuracy: 0.9396
Epoch 3/100
919/919 - 1s - loss: 0.1501 - accuracy: 0.9537 - val_loss: 0.1547 - val_accuracy: 0.9525
Epoch 4/100
919/919 - 1s - loss: 0.1221 - accuracy: 0.9634 - val_loss: 0.1445 - val_accuracy: 0.9562
Epoch 5/100
919/919 - 1s - loss: 0.1025 - accuracy: 0.9682 - val_loss: 0.1505 - val_accuracy: 0.9562
Epoch 6/100
919/919 - 1s - loss: 0.0837 - accuracy: 0.9727 - val_loss: 0.1462 - val_accuracy: 0.9589
Epoch 7/100
919/919 - 1s - loss: 0.0749 - accuracy: 0.9759 - val_loss: 0.1285 - val_accuracy: 0.9637
Epoch 8/100
919/919 - 1s - loss: 0.0635 - accuracy: 0.9796 - val_loss: 0.1366 - val_accuracy: 0.9635
Epoch 9/100
919/919 - 1s - loss: 0.0552 - accuracy: 0.9817 - val_loss: 0.1517 - val_accuracy: 0.9597
Epoch 10/100
919/919 - 1s - loss: 0.0453 - accuracy: 0.9858 - val_loss: 0.1395 - val_accura